# Predictive Model for Motorcycle Accidents:  to Insurance Premiums and Intersection Safety  (January 2020 - October 2024)

**Version 0.1.2**
#New Changes:
    -Added section: Data Cleaning
    -Updated queries for collision with full query
    -Added new queries to Data Cleaning



### **Project Overview**:

This report outlines a predictive model designed to determine motorcycle accident risk across NYC, based on intersection characteristics, finding patterns within individual characteristics, accident concentration by zip code, and commonalities among high risk areas. 

**Our Goal**: 

The model aims to aid insurance companies in adjusting premiums for high risk profiles and areas as well as provide government agencies with actionable data to improve traffic safety. 

We plan on doing so by analyzing historical accident data from the last 4 years, traffic patterns, and intersection structures. The predictive model will identify high risk areas and attempt to predict future accidents within a certain margin of error. 

**Actionable Insights**: 

Insurance policies could adjust premiums for high risk zip codes, while government agencies may enhance road safety measures, such as speed cameras and street widening, in these areas. 

**Methodology**: 

We plan on using a combination of a few programming languages to achieve our goal, we will be indicating when we use a specific one. 

We will be using SQL, and more specifically Google's RDBMS BigQuery and MySQL to retrieve and clean data. 

We will be using R to help with complex statistical analysis on the cleaned dataset. 

We will be using Python to be tying all the parts of the project together as well as for some extra analysis and regression.

# Data Collection:

To get our dataset we looked on the NYC public data in order to find the motorcycle accident data. 

We used this table for the personal profiles: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Person/f55k-p6yu/about_data

We used this table for the collision in the intersection: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/about_data

We used this table for the specific vehicles in the crashes: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Vehicles/bm4k-52h4/about_data

All of the data that we have collected will be stored into Google Cloud and we will use Google's BigQuery in order to query the data. 

Note: As of now, our data goes from January 1st 2020 to November 1st 2024. As the new year comes around, we will update the data so that it comprises all of 2020 to 2024 (including November and December of this year).

##^EDITORS NOTE USE INNER JOIN TO JOIN ANY MOTORCYCLE DATA

### Data Cleaning

Using BigQuery, we can use Structured Query Language (SQL) in order to query and clean our data. We will begin with the personal profiles table. 


In [ ]:
--SQL (BigQuery)
SELECT
  *
FROM
  `ferrous-thought-409502.insurance.people`
WHERE
  (CRASH_DATE BETWEEN '2020-01-01' AND '2024-10-01') 
  AND (PERSON_AGE IS NOT NULL 
  OR (PERSON_AGE >0 AND PERSON_AGE <150)) 
  AND PERSON_SEX IS NOT NULL


In order to get the aggregrated intersections which have the most amount of accidents, we will have to dive deeper. The following query will clean, aggregate, and display the intersections with the most amount of collisions.

We will also have an additional column that displays the total number of collisions as well as getting the latitude and longitude for each collision, this will help us when displaying our results. 

In [ ]:
--SQL (BigQuery)
SELECT 
  COUNT(DISTINCT COLLISION_ID) AS number_of_collisions, 
  LEAST(TRIM(UPPER(`ON STREET NAME`)), TRIM(UPPER(`CROSS STREET NAME`))) AS primary_street_name,
  GREATEST(TRIM(UPPER(`ON STREET NAME`)), TRIM(UPPER(`CROSS STREET NAME`))) AS secondary_street_name,
  LATITUDE as latitude,
  LONGITUDE as longitude,
  (SELECT COUNT (*) 
  FROM `ferrous-thought-409502.insurance.motorcycle` 
  ) AS total_accidents
FROM 
  `ferrous-thought-409502.insurance.motorcycle`
WHERE
  `CRASH DATE` BETWEEN '2020-01-01' AND '2024-11-01'
  AND BOROUGH IS NOT NULL
  AND `ZIP CODE` IS NOT NULL
  AND `ZIP CODE` >10000
  AND LATITUDE IS NOT NULL
  AND LONGITUDE IS NOT NULL
  AND `ON STREET NAME` IS NOT NULL
  AND `CROSS STREET NAME` IS NOT NULL
  AND TRIM(UPPER(`ON STREET NAME`)) <> ''
  AND TRIM(UPPER(`CROSS STREET NAME`)) <> ''
  AND TRIM(UPPER(`CROSS STREET NAME`)) <> TRIM(UPPER(`ON STREET NAME`))
  AND TRIM(UPPER(`ON STREET NAME`)) <> TRIM(UPPER(`CROSS STREET NAME`))

GROUP BY
  LEAST(TRIM(UPPER(`ON STREET NAME`)), TRIM(UPPER(`CROSS STREET NAME`))), 
  GREATEST(TRIM(UPPER(`ON STREET NAME`)), TRIM(UPPER(`CROSS STREET NAME`))),
  LATITUDE,
  LONGITUDE
ORDER BY
  COUNT(DISTINCT COLLISION_ID) DESC